In [ ]:
import pandas as pd
df = pd.read_csv('../data/hansard_archive/hansard_speeches.csv')
df = df[['speech', 'party']]
df.columns = ['doc', 'party']
df = df.dropna(subset = ['party'])
df = df[df['party'].isin(['Conservative', 'Labour'])]
df

In [ ]:
import sys
sys.path.append('../gtm/')
from utils import text_processor
    
p = text_processor('en_core_web_sm', pos_tags_to_keep = ['VERB', 'NOUN'])
df['doc_clean'] = p.process_docs(df['doc'])

In [ ]:
from collections import Counter
doc_clean = list(df['doc_clean'])
doc_clean_tokenized = [doc.split() for doc in doc_clean]
doc_clean_tokenized = [item for sublist in doc_clean_tokenized for item in sublist]
c = Counter(doc_clean_tokenized)

In [ ]:
c = {k:v for k,v in c.items() if v > 100}
print(len(c))
keepers = list(c)

doc_clean_tokenized = [doc.split() for doc in doc_clean]
docs = []
for doc in doc_clean_tokenized:
    d = []
    for tok in doc:
        if tok in keepers:
            d.append(tok)
    d = ' '.join(d)
    d = d.strip()
    docs.append(d)
df['doc_clean'] = docs
df = df[df["doc_clean"] != '']
df = df.reset_index(drop=True)

l = list(df.doc_clean)
idx = [k for k,v in enumerate(l) if len(v.split()) >= 20] # drop speeches of less than 20 preprocessed tokens
df = df.iloc[idx]
df = df.reset_index(drop=True)

df.to_csv('../data/hansard_archive/hansard_speeches_processed.csv')